## Runnable 的调用机制

### Runnable 基类实现了 __call__ 方法

几乎所有 illufly 的类将 Runnable 作为基类。

Runnable 自己实现了 __call__ 方法，并在这个方法中调用 call 方法。<br>
因为 call 是一个抽象方法，因此要求子类必须实现自己它。

有了 __call__ 方法，你就可以将类的实例当作方法一样使用。

**这样做的好处：**
这很方便，也足够简单，因为你只需要记住 illufly 中的智能体对象只有一个方法，并且你不需要记住名字。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        return "hi"

r = MyRun()
r()

hi


这样，Runnable 就可以通过 __call__ 方法调用自己的 call 方法。

### 使用 EventBlock 交换事件信息流

In [2]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield self.create_event_block("chunk", "hi")

r = MyRun()
r()

hi

## 使用 Runnable 的流式输出

### 默认的 log 处理

默认情况下，Runnable 的 handlers 列表中已经有 log 函数。
因此你可以隐藏调用 log 来打印流式内容。

In [3]:
# 使用 handler 函数的另一种方式：
from illufly.chat import FakeLLM

llm = FakeLLM()
llm("你能帮我写一首关于兔子做梦的四句儿歌?")

[AGENT] FakeLLM.4541258992
Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'

In [4]:
# 这与下面的代码等价
from illufly.io import log

llm = FakeLLM(handlers=[log])
llm("你能帮我写一首关于兔子做梦的四句儿歌?")

[AGENT] FakeLLM.4892204704
Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'

In [5]:
FakeLLM.allowed_params()

{'response': '响应内容',
 'sleep': '睡眠时间',
 'end_chk': '是否在最后输出一个 EndBlock',
 'fetching_context': '上下文提取标记，可通过修改环境变量 ILLUFLY_CONTEXT_START 和 ILLUFLY_CONTEXT_END 修改默认值',
 'fetching_final_answer': '最终答案提取标记，可通过修改环境变量 ILLUFLY_FINAL_ANSWER_START 和 ILLUFLY_FINAL_ANSWER_END 修改默认值',
 'fetching_output': '输出内容提取标记',
 'func': '用于自定义工具的同步执行函数',
 'async_func': '用于自定义工具的异步执行函数',
 'name': '工具名称',
 'handlers': 'EventBlock 迭代器处理函数列表，默认为 [log]，当调用 call 方法时，会使用该列表中的函数逐个处理 EventBlock',
 'block_processor': '在 yield 之前将 EventBlock 事件转换为新的格式，在 __call__ 方法的输出生成器时使用',
 'threads_group': '如果由 illufly 管理线程池实现并发或异步，则可以指定线程组名称，默认为 DEFAULT',
 'providers': '实例的 consumer_dict 属性由 providers 列表中每个 Runnable 的 provider_dict 属性提供',
 'consumers': '实例的 provider_dict 属性将被 consumers 列表中每个 Runnable 引用',
 'dynamic_providers': '如果实例在不同周期中重复使用，可能会希望先在绑定前先清除旧的绑定，此时就应该使用动态绑定，即执行 bind_provider 时提供 dynamic=True 参数',
 'lazy_binding_map': '有时你无法确定被哪个对象绑定，但能确定绑定映射，此时就可以使用 lazy_binding_map 参数，在绑定时由对方根据该参数进行绑定',
 'description': '工具描述',
 'tool

### 异步处理的 handler

只要 handler 中有一个是异步处理器，你就必须使用异步处理。<br>
alog 是 log 的异步版本。

In [1]:
from illufly.chat import FakeLLM
from illufly.io import alog

llm = FakeLLM(handlers=[alog])
await llm("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

[RUNNABLE] FakeLLM.4363828288
[USER] 你能帮我写一首关于兔子做梦的四句儿歌?
[AGENT] FakeLLM.4363828288
[INFO] 记住 10 轮对话
[INFO] I am FakeLLM
Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'

### `illufly.io` 中的 log 和 usage

usage 处理函数用于捕捉生成器结果中的 

In [ ]:
from illufly.chat import ChatOpenAI
from illufly.io import log, usage

openai = ChatOpenAI(handlers=[log, usage])
openai("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

## 使用绑定机制传播 handlers

In [3]:
from illufly.types import Runnable
from illufly.chat import FakeLLM
from illufly.io import log

### 定义一个新的 handler

定义 `handler` 必须使用形如 `block,, verbose, **kwargs` 这样的入参结构：

In [4]:
def myhandler(block, verbose=False, **kwargs):
    if block.block_type == 'info':
        print("I got a [info] event, but I wont tell you...")

In [5]:
llm = FakeLLM()
llm("hi")

[AGENT] FakeLLM.4715079488
Reply >> hi


'Reply >> hi'

### 将 provider 中的 handler 传递给 consumer

In [6]:
class myrun(Runnable):
    def call(self, prompt, **kwargs):
        llm = FakeLLM()
        self.bind_consumer(llm)
        resp = llm(prompt)
r = myrun(handlers = [log, myhandler])
r("hi")

[AGENT] FakeLLM.4735835664
I got a [info] event, but I wont tell you...
I got a [info] event, but I wont tell you...
Reply >> hi
None

In [8]:
r.consumer_tree

{'provider': <myrun.4715076032>,
 'consumer_tree': [{'consumer': <FakeLLM.4735835664>,
   'binding_map': {},
   'consumer_tree': {'provider': <FakeLLM.4735835664>, 'consumer_tree': []}}]}

In [9]:
llm.provider_tree

{'consumer': <FakeLLM.4715079488>, 'provider_tree': []}

## CallingEvent

illufly 支持复杂的事件流广播。

除了 ChatAgent 调用大模型接口引发的流输出，还有 RAG 检索、工具回调等中间信息，以及使用 FlowAgent、ChatAgent、BaseAgent 之间的相互嵌套，都会导致多个来源的 EventBlock 爆发。

如同在 ChatAgent 中允许手动管理 **thread_id**，Runnable 基类允许手动管理 **calling_id** ，并提供基于 calling_id 的汇聚管理。

这可以支撑在 Web 客户端按照某 Agent 、某连续对话过程、某次调用来呈现信息流，实现最佳用户体验。

### 默认 generator

In [2]:
# 使用 handler 函数的另一种方式：
from illufly.chat import FakeLLM, ChatQwen

llm = ChatQwen(handlers=[])
for b in llm("你能帮我写一首关于兔子做梦的四句儿歌?", generator="sync"):
    print(b)

event: runnable
data: ChatQwen.4348625920

event: user
data: 你能帮我写一首关于兔子做梦的四句儿歌?

event: agent
data: ChatQwen.4348625920

event: info
data: 记住 10 轮对话

event: chunk
data: 当然

event: chunk
data: 可以

event: chunk
data: ，

event: chunk
data: 希望

event: chunk
data: 下面这首小诗

event: chunk
data: 能够满足您的需求

event: chunk
data: ：

小白兔蹦

event: chunk
data: 又跳，
梦

event: chunk
data: 里花儿笑

event: chunk
data: 。
月亮船轻轻

event: chunk
data: 摇，
甜蜜梦

event: chunk
data: 乡绕。

event: chunk
data: 

event: new_line
data: 

event: usage
data: {"input_tokens": 21, "output_tokens": 35, "total_tokens": 56}

event: final_text
data: 当然可以，希望下面这首小诗能够满足您的需求：

小白兔蹦又跳，
梦里花儿笑。
月亮船轻轻摇，
甜蜜梦乡绕。



### 异步生成器

In [1]:
# 使用 handler 函数的另一种方式：
from illufly.chat import FakeLLM, ChatQwen

llm = ChatQwen(handlers=[])
async for b in llm("你能帮我写一首关于兔子做梦的四句儿歌?", generator="async"):
    print(b)

event: runnable
data: ChatQwen.4348051360

event: user
data: 你能帮我写一首关于兔子做梦的四句儿歌?

event: agent
data: ChatQwen.4348051360

event: info
data: 记住 10 轮对话

event: chunk
data: 小白

event: chunk
data: 兔

event: chunk
data: 入

event: chunk
data: 梦

event: chunk
data: 乡，  
梦

event: chunk
data: 中花儿香

event: chunk
data: 。  
跳过

event: chunk
data: 彩虹桥，  


event: chunk
data: 找着月亮糖

event: chunk
data: 。

event: chunk
data: 

event: new_line
data: 

event: usage
data: {"input_tokens": 21, "output_tokens": 25, "total_tokens": 46}

event: final_text
data: 小白兔入梦乡，  
梦中花儿香。  
跳过彩虹桥，  
找着月亮糖。



### 自定义 block_processor

In [1]:
# 使用 handler 函数的另一种方式：
from illufly.chat import FakeLLM, ChatQwen

blocks = []
def handle(block, **kwargs):
    blocks.append(block.text)
    return block.text

llm = ChatQwen(block_processor=handle)
for b in llm("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True, generator=True):
    pass
blocks
# llm("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

[RUNNABLE] ChatQwen.4386570832
[USER] 你能帮我写一首关于兔子做梦的四句儿歌?
[AGENT] ChatQwen.4386570832
[INFO] 记住 10 轮对话
当然可以，希望你喜欢这首儿歌：

小兔乖乖睡梦乡，  
梦中草儿香又长。  
蹦蹦跳跳乐无疆，  
醒来依旧喜洋洋。
[USAGE] {"input_tokens": 21, "output_tokens": 40, "total_tokens": 61}


['ChatQwen.4386570832',
 '你能帮我写一首关于兔子做梦的四句儿歌?',
 'ChatQwen.4386570832',
 '记住 10 轮对话',
 '当然',
 '可以',
 '，',
 '希望',
 '你喜欢这首儿歌',
 '：\n\n小兔乖乖',
 '睡梦乡，',
 '  \n梦中草',
 '儿香又长',
 '。  \n蹦蹦',
 '跳跳乐无',
 '疆，  \n醒来',
 '依旧喜洋洋。',
 '',
 '',
 '{"input_tokens": 21, "output_tokens": 40, "total_tokens": 61}',
 '当然可以，希望你喜欢这首儿歌：\n\n小兔乖乖睡梦乡，  \n梦中草儿香又长。  \n蹦蹦跳跳乐无疆，  \n醒来依旧喜洋洋。']